In [1]:
import numpy as np
import pandas as pd
import random
from croston import croston
import matplotlib.pyplot as plt
from datetime import datetime
from datetime import timedelta

In [2]:
def read_data(source):
    df = pd.read_csv(source)
    
    df = df[:2322]
    
    df['Date'] = pd.to_datetime(df['pay_date'])
    df.index = df['Date']
    del df['Date'] # , df['pay_date'], 
    df.sort_index()
    
    #df = df.groupby(["prod_uid"])
    
    return df

#read dataframe
df = read_data("daily_67_amount.csv")

In [17]:
pred_length = 4
input_data = df.loc[:datetime.strptime("2021-01-05", "%Y-%m-%d")]
fit_pred = croston.fit_croston(input_data["amount"], pred_length,'original')
fit_pred

{'croston_model': {'a_demand': 0.9978125000000031,
  'a_interval': 0.9978125000000031,
  'demand_series': 0             1
  1             1
  2             1
  3       1.99781
  4             2
           ...   
  1350     1.0022
  1351          1
  1352          1
  1353    2.99563
  1354    4.99562
  Length: 1355, dtype: object,
  'interval_series': 0       1.65683
  1       1.00144
  2       4.99125
  3       1.00873
  4       6.98689
           ...   
  1350    3.99344
  1351    2.00436
  1352    2.99782
  1353    2.00218
  1354    1.00219
  Length: 1355, dtype: object,
  'demand_process': 0       0.603563
  1       0.998565
  2        0.20035
  3        1.98052
  4        0.28625
            ...   
  1350    0.250962
  1351    0.498915
  1352    0.333575
  1353     1.49618
  1354     4.98469
  Length: 1355, dtype: object,
  'correction_factor': 1},
 'croston_fittedvalues': array([0.        , 0.        , 0.        , ..., 0.3335755 , 1.49617954,
        4.98468713]),
 'croston_forec

In [3]:
pred_length = 4
datelist = pd.date_range(datetime.strptime("2021-01-05", "%Y-%m-%d"), datetime.strptime("2021-03-22", "%Y-%m-%d")).tolist()
x = {'begin Date':[],
     'Monday':[],
     'Mon_true':[],
     'Wednesday':[],
     'Wed_true':[],
     'Friday':[],
     'Fri_true':[],
    }

for i in datelist:
    if i.dayofweek == 1:
        input_data = df.loc[:i]
        fit_pred = croston.fit_croston(input_data["amount"], pred_length,'original')
        yhat = pd.DataFrame(np.concatenate([fit_pred['croston_fittedvalues'], fit_pred['croston_forecast']]))
        predict = yhat[-3:-1].sum()
        true = df.loc[i+timedelta(days=2) : i+timedelta(days=3)]
        
        x['begin Date'].append(i)
        x['Monday'].append(int(predict))
        x['Mon_true'].append(sum(true['amount']))
        
    elif i.dayofweek == 3:
        input_data = df.loc[:i]
        fit_pred = croston.fit_croston(input_data["amount"], pred_length,'original')
        yhat = pd.DataFrame(np.concatenate([fit_pred['croston_fittedvalues'], fit_pred['croston_forecast']]))
        predict = yhat[-3:].sum()
        true = df.loc[i+timedelta(days=2) : i+timedelta(days=4)]
        
        x['Wednesday'].append(int(predict))
        x['Wed_true'].append(sum(true['amount']))
        
    elif i.dayofweek == 5:
        input_data = df.loc[:i]
        fit_pred = croston.fit_croston(input_data["amount"], pred_length,'original')
        yhat = pd.DataFrame(np.concatenate([fit_pred['croston_fittedvalues'], fit_pred['croston_forecast']]))
        predict = yhat[-2:].sum()
        true = df.loc[i+timedelta(days=3) : i+timedelta(days=4)]
        
        x['Friday'].append(int(predict))
        x['Fri_true'].append(sum(true['amount']))
        

In [4]:
result = pd.DataFrame.from_dict(x)

In [5]:
import math
error = []
mse = 0
for i in range(len(result)):
    a = result.loc[i, 'Monday'] + result.loc[i, 'Wednesday'] + result.loc[i, 'Friday']
    b = result.loc[i, 'Mon_true'] + result.loc[i, 'Wed_true'] + result.loc[i, 'Fri_true']
    c = (a - b)/a
    mse = mse + pow((a - b), 2)
    error.append(c)

In [8]:
result['error'] = error
print("MSE = ", mse/len(result))
result

MSE =  553.9090909090909


,begin Date,Monday,Mon_true,Wednesday,Wed_true,Friday,Fri_true,error
0,2021-01-05,9,3,9,0,6,0,0.875000
1,2021-01-12,6,13,5,0,2,0,0.000000
2,2021-01-19,2,1,3,6,0,9,-2.200000
3,2021-01-26,3,8,12,13,8,4,-0.086957
4,2021-02-02,6,9,5,4,13,11,0.000000
5,2021-02-09,21,0,32,0,21,5,0.932432
6,2021-02-16,21,6,14,5,7,12,0.452381
7,2021-02-23,21,3,5,0,2,9,0.571429
8,2021-03-02,2,1,3,3,2,5,-0.285714
9,2021-03-09,9,10,7,2,9,1,0.480000
